# Setup & Import

In [1]:
import random
import nltk
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import codecs

In [2]:
from google.colab import drive
drive.mount('/content/drive/')
pathNameData = '/content/drive/MyDrive/Datasets/'

Mounted at /content/drive/


# Load Data

In [9]:
# BOT_CONFIG = {
#     'intents': {
#         'hello': {
#             'examples': ['Привет!', 'Здарова', 'Хей-хей!!'],
#             'responses': ['Хай', 'Добрый вечер!', 'Здравствуйте!']
#         },
#         'bye': {
#             'examples': ['Пока', 'Увидимся!', 'Покеда'],
#             'responses': ['До свидания', 'Прощайте', 'Сайонара!']
#         }
#     }
# }
#with open('/content/chat_bot_config.json', 'r') as f:
import codecs
with codecs.open(pathNameData + 'BOT_CONFIG1.json', 'r', 'utf-8') as f:
  BOT_CONFIG = json.load(f)
print(len(list(BOT_CONFIG['intents'])))

244


In [15]:
'unique intents:', BOT_CONFIG['intents'].keys()

('unique intents:',
 dict_keys(['hello', 'bye', 'where_from', 'how_are_you', 'yo', 'music', 'whereyoufrom', 'myfood', 'whether', 'condition', 'sport', 'whats_up', 'what_are_you_doing', 'weather1', 'weather2', 'howareyou', 'book', 'games', 'affairs', 'youhere', 'youfrom', 'functional', 'about_where_job', 'about_cats', 'about_favorite_movie', 'about_wise', 'random', 'agree', 'empty', 'cinemaposter', 'whatfordinner', 'musicforrunning', 'elephantweight', 'whatcaniask', "what's up", 'favorite color', 'vacation', 'how', 'talking', 'busy', 'weather', 'name', 'age', 'help', 'skillbox', 'mood', 'possibilities', 'jokes', 'route', 'love', 'how_do_you_do', 'what_do_you_want', 'how_old', 'family', 'health', 'knowledge', 'ideas', 'elections', 'how_do_yo_do', 'doing', 'day', 'wannaeat', 'areyousmart', 'time', 'status', 'advice', 'employment', 'philosophic', 'food', 'news', 'funny', 'eat', 'drink', 'busyness', 'plans', 'wearther', 'depressed', 'whatson', 'cost', 'booking_days', 'pays', 'photos', 'info

In [14]:
BOT_CONFIG['intents']

{'about_air': {'examples': ['что такое воздух',
   'расскажи про воздух',
   'знаешь что-нибудь о воздухе'],
  'responses': ['смесь газов образующих земную атмосферу',
   'воздух необходим для нормального существования на земле живых организмов',
   'это одна из четырёх стихий космоса']},
 'about_cats': {'examples': ['любишь кошек',
   'нравятся кошки',
   'кошки нравятся',
   'любишь котов'],
  'responses': ['кошки топ', 'люблю больше чем людей', 'очень']},
 'about_earth': {'examples': ['что такое земля',
   'расскажи про землю',
   'знаешь что-нибудь о земле'],
  'responses': ['третья по удалённости от солнца планета солнечной системы',
   'пятая по диаметру и массе среди всех планет',
   'единственная населённое живыми организмами']},
 'about_favorite_movie': {'examples': ['любимое кино',
   'любимый фильм',
   'какой твой самый любимый фильм',
   'какое твое самое любимое кино'],
  'responses': ['тот фильм, где роботы победили', 'терминатор', 'матрица']},
 'about_water': {'examples

In [ ]:
#with open(pathNameData + 'BOT_CONFIG1.json', 'w') as f:
   #json.dump(BOT_CONFIG, f, ensure_ascii=False, indent=3) # для русских текстов]

# Prepare Data

## Utils

In [26]:
rus_lower_letters = ''.join([chr(i) for i in range(ord('а'),ord('а')+32)])
def clean(text):
  text = text.lower()
  cleaned_text = ''
  for ch in text:
    if ch in rus_lower_letters:
      cleaned_text = cleaned_text + ch
  return cleaned_text

In [ ]:
# вариант 1 
def get_intent(text): 
  for intent in BOT_CONFIG['intents'].keys():
    #for example in BOT_CONFIG['intents'][intent]['examples']:
    for example in intent['examples']:
      w1 = clean(example)
      w2 = clean(text)
      if nltk.edit_distance(w1, w2) / max(len(w1), len(w2)) < 0.4:
        return intent
  return 'интент не найден'

## Prepare inputs, targets

In [17]:
X = []
y = []

for i, intent in enumerate(BOT_CONFIG['intents'].items()):
    try:
        for example in intent[1]['examples']:
            X.append(example)
            y.append(intent[0])
    except:
        pass

len(X), len(y)

(841, 841)

In [18]:
print(X); print(y)

['привет!', 'здарова', 'хей-хей!!', 'пока', 'увидимся!', 'покеда', 'ты откуда?', 'где живешь?', 'не местный?', 'как дела?', 'как ты?', 'как поживаешь?', 'йоу', 'хой', 'псс..', 'любишь музыку?', 'слушаешь музыку?', 'любимая песня?', 'любимая группа', 'слышал эту песню?', 'любишь слушать музыку?', 'любишь эту группу?', 'ты откуда?', 'где живешь?', 'из какого ты города?', 'где ты находишься?', 'в каком городе живешь?', 'из какой страны?', 'где твой дом?', 'что ты ешь?', 'что ты любишь есть?', 'какое любимое блюдо?', 'твоё любимое блюдо', 'чем ты питаешься?', 'что ты обычно ешь?', 'ты любишь есть?', 'погода', 'температура', 'влажность', 'как дела?', 'как самочувствие?', 'как себя чувствуешь?', 'как матч?', 'кто выигрывает', 'за кого болеешь', 'как дела?', 'что настроение?', 'как сам?', 'что делаешь?', 'чем занят?', 'чем занимаешься?', 'как тебе погода?', 'что думаешь о погоде?', 'погода', 'тебе нравится дождь?', 'что думаешь о теплой погоде?', 'что думаешь о холодной погоде?', 'как жизнь?'

### Split Train Test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)
len(X_train), len(X_test)

(672, 169)

### Vectorize Data

In [27]:
%%time
from sklearn.feature_extraction.text import CountVectorizer #, TfidfVectorizer
# вариант 1
# vectorizer = CountVectorizer()
# X_vect = vectorizer.fit_transform(X)

# варинат 2
vectorizer = CountVectorizer(preprocessor=clean, analyzer='char', ngram_range=(2,3))
X_vect = vectorizer.fit_transform(X)
print( len(vectorizer.get_feature_names()), "\n", vectorizer.get_feature_names()[:10], "\n", vectorizer.get_feature_names()[-10:])

2944 
 ['аа', 'ааа', 'аан', 'аб', 'аба', 'абл', 'або', 'абр', 'абс', 'абу'] 
 ['ях', 'яха', 'яхо', 'яц', 'яце', 'яч', 'ячт', 'ячу', 'ящ', 'яще']
CPU times: user 33.9 ms, sys: 0 ns, total: 33.9 ms
Wall time: 39.3 ms


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [29]:
print(len(vectorizer.vocabulary_), "\n", vectorizer.vocabulary_)
word_num_sorted = sorted(vectorizer.vocabulary_.items(), key=lambda t: t[1], reverse=True)
word_index = dict((c[0], i) for i, c in enumerate(word_num_sorted))
index_word = dict((i, c[0]) for i, c in enumerate(word_num_sorted))

print(word_nums)
print(word_index)
print(index_word)

2944 
 {'пр': 1864, 'ри': 1919, 'ив': 861, 'ве': 290, 'ет': 701, 'при': 1867, 'рив': 1921, 'иве': 864, 'вет': 299, 'зд': 808, 'да': 441, 'ар': 123, 'ро': 1942, 'ов': 1593, 'ва': 271, 'зда': 809, 'дар': 453, 'аро': 129, 'ров': 1944, 'ова': 1594, 'хе': 2442, 'ей': 597, 'йх': 1091, 'хей': 2443, 'ейх': 601, 'йхе': 1092, 'по': 1841, 'ок': 1686, 'ка': 1102, 'пок': 1851, 'ока': 1687, 'ув': 2304, 'ви': 304, 'ид': 876, 'ди': 478, 'им': 935, 'мс': 1413, 'ся': 2130, 'уви': 2307, 'вид': 305, 'иди': 879, 'дим': 482, 'имс': 943, 'мся': 1418, 'ке': 1125, 'ед': 564, 'оке': 1688, 'кед': 1127, 'еда': 565, 'ты': 2251, 'ыо': 2679, 'от': 1764, 'тк': 2193, 'ку': 1194, 'уд': 2314, 'тыо': 2264, 'ыот': 2684, 'отк': 1771, 'тку': 2197, 'куд': 1195, 'уда': 2315, 'гд': 395, 'де': 460, 'еж': 580, 'жи': 764, 'еш': 735, 'шь': 2587, 'где': 397, 'деж': 463, 'ежи': 583, 'жив': 766, 'веш': 300, 'ешь': 739, 'не': 1466, 'ем': 630, 'ме': 1359, 'ес': 685, 'ст': 2099, 'тн': 2201, 'ны': 1558, 'ый': 2648, 'нем': 1477, 'еме': 63

### Split Dataset

In [30]:
X_vect_train, X_vect_test, y_train, y_test = train_test_split(
    X_vect, y, test_size=0.2, random_state=42)

In [31]:
X_vect_train.shape, X_vect_test.shape

((672, 2944), (169, 2944))

## Create, Train and Evaluate Model

### Regression

In [32]:
log_reg = LogisticRegression()

In [47]:
%%time
#log_reg.fit(X_vect, y)
log_reg.fit(X_vect_train, y_train)

CPU times: user 10.2 s, sys: 8.28 s, total: 18.5 s
Wall time: 9.78 s


LogisticRegression()

In [49]:
#print(log_reg.score(X_vect, y))
print(log_reg.score(X_vect_train, y_train))
print(log_reg.score(X_vect_test, y_test))

0.8377976190476191
0.17751479289940827


### Classification

In [46]:
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import PassiveAggressiveRegressor
clf = PassiveAggressiveClassifier()
clf.fit(X_vect_train, y_train)
print(clf.score(X_vect_train, y_train))
print(clf.score(X_vect_test, y_test))

0.8258928571428571
0.1834319526627219


## Create Bot

In [23]:
def get_intent_by_model2(model, text):
    
    if 'Classifier' in str(type(model)):
      pred = model.predict(vectorizer.transform([text]))[0]
    else: # Regressor
      pred = model.predict(vectorizer.transform([text]))[0]
    return pred

In [51]:
def bot2(model, question):
  intent_0 = get_intent_by_model2(model, question)
  if 'Classifier' in str(type(model)):
      
      #intent = y_train[intent_0]
      intent = intent_0
  elif 'Regression' in str(type(model)):
    intent = intent_0
  else:
    return "с такой моделью я не работаю"
  print(f"///intent: '{intent}'")
  return random.choice(BOT_CONFIG['intents'][intent]['responses'])

## Predict (Regression)

In [25]:
stop_list = ['стоп']

question = ''
while question not in stop_list:
  question = input('Ваш вопрос?: ')
  if question  not in stop_list:
      answer = bot2(log_reg, question)
      print('-', answer)
  else:
    print("...by")

Ваш вопрос?: Как дела
///intent: 'how are you'
- лучше и быть не может!
Ваш вопрос?: Как погода
///intent: 'wearther'
- все в пределах климатической нормы
Ваш вопрос?: Что новенького
///intent: 'what's up'
- ничего особенного.
Ваш вопрос?: Что думаете о завтрашнем
///intent: 'what's up'
-  как обычно.
Ваш вопрос?: Как здоровье
///intent: 'health'
- прощайте-я помахал платочком в след здоровью!
Ваш вопрос?: откуда вы
///intent: 'busy'
- слушаю вас
Ваш вопрос?: Откуда
///intent: 'whereyoufrom'
- виртуальная реальность мой дом!
Ваш вопрос?: что ешь обычно
///intent: 'myfood'
- я же бот, я не ем
Ваш вопрос?: как тебе работа
///intent: 'work'
- отлично!
Ваш вопрос?: какую музыку любишь
///intent: 'music'
- музыка для людей
Ваш вопрос?: а что в спорет нравится
///intent: 'what's up'
-  как обычно.
Ваш вопрос?: а в спорте что
///intent: 'what's up'
- ничего особенного.
Ваш вопрос?: соревнования любишь
///intent: 'music'
- музыка для людей
Ваш вопрос?: откуда ты
///intent: 'whereyoufrom'
- вир

## Predict (Classification)

In [52]:
stop_list = ['стоп']

question = ''
while question not in stop_list:
  question = input('Ваш вопрос?: ')
  if question  not in stop_list:
      answer = bot2(clf, question)
      print('-', answer)
  else:
    print("...by")

Ваш вопрос?: привет
///intent: 'hello'
- добрый вечер!
Ваш вопрос?: как дела
///intent: 'affairs'
- не дождетесь!
Ваш вопрос?: что-то боитесь
///intent: 'appearance'
- что за вопросы, товарищ майор?
Ваш вопрос?: очень интересно
///intent: 'plans'
- надо подумать...
Ваш вопрос?: что любите
///intent: 'whatdoyoulike'
- python
Ваш вопрос?: а музыка
///intent: 'creativity'
- мгахи им. сурикова!
Ваш вопрос?: какие песни нравятс
///intent: 'whatson'
- люди такие беспокойные! у них все время что-то происходит...
Ваш вопрос?: спорт
///intent: 'tomato'
- воспользуйся ножом для чистки овощей
Ваш вопрос?: овощи?
///intent: 'bad'
- жаль
Ваш вопрос?: овощи
///intent: 'bad'
- сочувствую
Ваш вопрос?: стоп
...by
